## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import numpy as np 
import pandas as pd 
import re
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
import joblib
import seaborn as sns

### 1. Data Loading

In [2]:
class DataLoader:
  def __init__(self,dir_name,file_name):
    self.dir_name = dir_name
    self.file_name = file_name
    
 
  def read_csv(self):
    os.chdir(self.dir_name)
    tweets_df=pd.read_csv(self.file_name)
    return tweets_df

data_loader_object = DataLoader('/content/drive/MyDrive/', 'Week1_challenge_data_source(CSV).csv')
raw_data = data_loader_object.read_csv()

### .

##### Task 3.1 Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
* Average TCP retransmission
* Average RTT
* Handset type
* Average throughput


In [3]:
raw_data.fillna(raw_data.mean(), inplace=True)
columns = ["MSISDN/Number", "Average RTT", "Average TCP retransmission", "Average throughput"]
data_filled = pd.DataFrame(columns=columns)

In [4]:
data_filled["MSISDN/Number"] = raw_data.groupby("MSISDN/Number").agg({"Bearer Id": "count"}).reset_index()["MSISDN/Number"]
data_filled["Average RTT"] = raw_data.groupby("MSISDN/Number").agg({"Avg RTT DL (ms)": "mean"}).reset_index()["Avg RTT DL (ms)"]+ raw_data.groupby("MSISDN/Number").agg({"Avg RTT UL (ms)": "sum"}).reset_index()["Avg RTT UL (ms)"]
data_filled["Average TCP retransmission"] = raw_data.groupby("MSISDN/Number").agg({"TCP DL Retrans. Vol (Bytes)": "mean"}).reset_index()["TCP DL Retrans. Vol (Bytes)"]+ raw_data.groupby("MSISDN/Number").agg({"TCP UL Retrans. Vol (Bytes)": "sum"}).reset_index()["TCP UL Retrans. Vol (Bytes)"]
data_filled["Average throughput"] = raw_data.groupby("MSISDN/Number").agg({"Avg Bearer TP DL (kbps)": "mean"}).reset_index()["Avg Bearer TP DL (kbps)"]+ raw_data.groupby("MSISDN/Number").agg({"Avg Bearer TP UL (kbps)": "sum"}).reset_index()["Avg Bearer TP UL (kbps)"]

##### Task 3.2 - Compute & list 10 of the top, bottom and most frequent:
1. TCP values in the dataset. 
2. RTT values in the dataset.
3. Throughput values in the dataset.


In [5]:
ten_most_tcp = data_filled.nlargest(n=10, columns=["Average TCP retransmission"])["Average TCP retransmission"]
ten_bottom_tcp = data_filled.nsmallest(n=10, columns=["Average TCP retransmission"])["Average TCP retransmission"]

ten_most_rtt = data_filled.nlargest(n=10, columns=["Average RTT"])["Average RTT"]
ten_bottom_rtt = data_filled.nsmallest(n=10, columns=["Average RTT"])["Average RTT"]

ten_most_tp = data_filled.nlargest(n=10, columns=["Average throughput"])["Average throughput"]
ten_bottom_tp = data_filled.nsmallest(n=10, columns=["Average throughput"])["Average throughput"]


In [6]:
data_filled.head()

,MSISDN/Number,Average RTT,Average TCP retransmission,Average throughput
0,3.360100e+10,46.000000,2.156957e+07,76.0
1,3.360100e+10,31.000000,2.156957e+07,99.0
2,3.360100e+10,127.458589,2.156957e+07,97.0
3,3.360101e+10,84.000000,7.607247e+05,248.0
4,3.360101e+10,62.000000,1.586063e+07,36646.5


##### Task 3.3 - Compute & report:
1. The distribution of the average throughput  per handset type and provide interpretation for your findings.
2. The average TCP retransmission view per handset type and provide interpretation for your findings.


##### Task 3.4 - Using the experience metrics above, perform a k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. (The description must define each group based on your understanding of the data)

In [7]:
from sklearn.preprocessing import MinMaxScaler
def normalizer(df):
  norm = MinMaxScaler()
  # normalize the exponential data with boxcox
  normalized_data = norm.fit_transform(df)
  return normalized_data
new_data = normalizer(data_filled[1:])

In [8]:
from sklearn.cluster import KMeans
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(new_data)

In [9]:
experience_centriods = pd.DataFrame(kmeans.cluster_centers_)

In [10]:
normalized_data = pd.DataFrame(new_data)

In [12]:
experience_centriods.to_csv("experience_centriods.csv", index=False)
normalized_data.to_csv("experience_data.csv", index=False)